## 移除最后conv最后一层网络


In [1]:
import sys
import os
import time
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.image as mping
import torch
import torch.nn as nn
import torchvision.models as models
import torch.utils.data as data
from torchvision import transforms

from data_loader import get_loader
from data_loader import get_encoder_loader
from model import *

%matplotlib inline
%config InlineBackend.figure_format = 'retina'

### set high parameters

In [4]:
batch_size = 256
num_epochs = 200
extract_size = 1024
class_size = 61


# 图片格式转化
transform_train = transforms.Compose([
    transforms.Resize([224,224]),
#     transforms.RandomCrop(224),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225])
])
transform_vaild = transforms.Compose([
    transforms.Resize([224,224]),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225])
])

### load encoded datas

In [3]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# exract the images to embedding tensor
# remove the last conv layer
encoder = EncoderCut()
encoder = encoder.to(device)

data_loader = get_encoder_loader(transform_train, encoder, device, mode='train', batch_size=batch_size)
data_loader.dataset.save_to("./bottle_neck/resnet152_train_remove_last_conv.h")
# data_loader = get_encoder_loader(transform_train, encoder, device, mode='train', batch_size=batch_size, file='./bottle_neck/resnet152_train_no_crop.h')

encoding 32738/32739->100.00%, spent_time:40:38.01

### init model

使用两层网络,结果过拟合,训练准确率很高,验证结果比较差,使用再试

In [5]:
# set the total number of training steps per epoch
total_step = int(len(data_loader.dataset)/batch_size)

classify_model = FinalClassify(extract_size, class_size)
classify_model = classify_model.to(device)

criterion = nn.CrossEntropyLoss().cuda() if torch.cuda.is_available() else nn.CrossEntropyLoss()

# with RMSprop to slow the desent gradient progress
optimizer = torch.optim.Adam(classify_model.parameters(), lr=0.001)

### Load the best trained model,yet!

In [21]:
classify_model.load_state_dict(torch.load('./models/class_single_model_last.pkl'))

### time to train model

In [9]:
optimizer = torch.optim.SGD(classify_model.parameters(), lr=0.0001)

In [10]:
classify_model = classify_model.train()
best_acc = 0

for epoch in range(201, num_epochs+201):
    start = time.time()
    for i_step in range(1, total_step+1):
        
        # Ramdomly get samples
        indices = data_loader.dataset.get_train_indices()
        new_sampler = data.sampler.SubsetRandomSampler(indices=indices)
        data_loader.batch_sampler.sampler = new_sampler
        
        embeds, targets = next(iter(data_loader))
        
        embeds = embeds.squeeze(1)
        targets = targets.type(torch.LongTensor).to(device)
        
        classify_model.zero_grad()
        
        outputs = classify_model(embeds)
        
        loss = criterion(outputs, targets.view(-1))
        
        loss.backward()
        
        optimizer.step()
        
        if i_step%20 == 0:
            # calculate the status
            predict_result = outputs.argmax(1)
            accuracy = torch.sum(predict_result == targets).item() / batch_size * 100
            best_acc = accuracy if accuracy > best_acc else best_acc
            
            stats = 'Epoch [%d/%d], Step [%d/%d], Loss: %.4f, Accuracy: %.2f%%, Best_acc: %.2f%%\
            ' % (epoch, num_epochs, i_step, total_step, loss.item(), accuracy, best_acc)
            print('\r' + stats, end='')
            sys.stdout.flush()
    print('\n Epoch {}, spent time:{:.2f}s'.format(epoch, time.time()-start))       
    if epoch%50 == 0:
        torch.save(classify_model.state_dict(), os.path.join('./models', 'class_single_rm1layer_%d.pkl' % epoch))
torch.save(classify_model.state_dict(), os.path.join('./models', 'class_single_rm2layer_last.pkl'))

Epoch [101/100], Step [120/127], Loss: 1.0080, Accuracy: 73.83%, Best_acc: 78.12%            
 Epoch 101, spent time:5.82s
Epoch [102/100], Step [120/127], Loss: 0.9286, Accuracy: 78.91%, Best_acc: 78.91%            
 Epoch 102, spent time:5.85s
Epoch [103/100], Step [120/127], Loss: 0.9704, Accuracy: 78.52%, Best_acc: 78.91%            
 Epoch 103, spent time:5.77s
Epoch [104/100], Step [120/127], Loss: 1.0021, Accuracy: 73.83%, Best_acc: 83.59%            
 Epoch 104, spent time:5.82s
Epoch [105/100], Step [120/127], Loss: 1.0476, Accuracy: 75.78%, Best_acc: 83.59%            
 Epoch 105, spent time:5.85s
Epoch [106/100], Step [120/127], Loss: 0.9959, Accuracy: 78.52%, Best_acc: 83.59%            
 Epoch 106, spent time:5.82s
Epoch [107/100], Step [120/127], Loss: 1.0700, Accuracy: 69.53%, Best_acc: 83.59%            
 Epoch 107, spent time:5.83s
Epoch [108/100], Step [120/127], Loss: 1.0042, Accuracy: 73.44%, Best_acc: 83.59%            
 Epoch 108, spent time:5.86s
Epoch [109/100],

Epoch [167/100], Step [120/127], Loss: 1.0286, Accuracy: 74.61%, Best_acc: 83.59%            
 Epoch 167, spent time:5.85s
Epoch [168/100], Step [120/127], Loss: 0.9817, Accuracy: 74.22%, Best_acc: 83.59%            
 Epoch 168, spent time:5.83s
Epoch [169/100], Step [120/127], Loss: 0.9718, Accuracy: 74.22%, Best_acc: 83.59%            
 Epoch 169, spent time:5.76s
Epoch [170/100], Step [120/127], Loss: 0.9926, Accuracy: 75.78%, Best_acc: 83.59%            
 Epoch 170, spent time:5.84s
Epoch [171/100], Step [120/127], Loss: 0.9966, Accuracy: 76.17%, Best_acc: 83.59%            
 Epoch 171, spent time:5.85s
Epoch [172/100], Step [120/127], Loss: 0.8985, Accuracy: 76.56%, Best_acc: 83.59%            
 Epoch 172, spent time:5.80s
Epoch [173/100], Step [120/127], Loss: 0.9745, Accuracy: 74.22%, Best_acc: 83.59%            
 Epoch 173, spent time:5.79s
Epoch [174/100], Step [120/127], Loss: 0.9901, Accuracy: 78.12%, Best_acc: 83.59%            
 Epoch 174, spent time:5.80s
Epoch [175/100],

### How good is the model?

In [7]:
# valid_data_loader = get_encoder_loader(transform_vaild, encoder, device, mode='valid', batch_size=batch_size)
# valid_data_loader.dataset.save_to("./bottle_neck/resnet152_valid_rm1layer.h")
valid_data_loader = get_encoder_loader(transform_vaild, encoder, device, mode='valid', batch_size=batch_size, file="./bottle_neck/resnet152_valid_rm1layer.h")

encoding 4981/4982->99.98%, spent_time:5:14.77

In [29]:
classify_model.load_state_dict(torch.load('./models/class_single_rm1layer_last.pkl'))

In [11]:
classify_model = classify_model.eval()
predict = []
for embed, _ in valid_data_loader.dataset:
    p = classify_model(embed).argmax().item()
    predict.append(p)
    
df_refer = valid_data_loader.dataset.refer
df_refer["predict"] = predict
df_refer['correct'] = df_refer.predict == df_refer.disease_class
accuracy = (df_refer.correct == True).sum()/len(df_refer)
print('The final accuracy is %.2f%%.' % (accuracy*100))

The final accuracy is 70.39%.
